In [ ]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime
import gc
import re

# File Reading

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string)
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting testutility.py


In [ ]:
%%writefile file.yaml
file_type: csv
dataset_name: testfile
file_name: test_data
table_name: edsurv
inbound_delimiter: ";"
outbound_delimiter: "|"
skip_leading_rows: 1
columns:
    - Period
    - Data
    - Magnitude
    - Subject
    - Group
    - Series



Overwriting file.yaml


In [ ]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [ ]:
config_data['inbound_delimiter']

';'

In [ ]:
#inspecting data of config file
config_data

{'file_type': 'csv',
 'dataset_name': 'testfile',
 'file_name': 'test_data',
 'table_name': 'edsurv',
 'inbound_delimiter': ';',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['Period', 'Data', 'Magnitude', 'Subject', 'Group', 'Series']}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
print(os.listdir())

['.config', 'file.yaml', '__pycache__', 'testutility.py', 'final_animedataset.csv', 'drive', 'sample_data']


In [ ]:
import pandas as pd

try:
    # Read the CSV file with the correct delimiter (tab '\t')
    csv_file_path = "/content/drive/MyDrive/final_animedataset.csv"
    df_sample = pd.read_csv(csv_file_path, delimiter='\t')

    # Display the first few rows of the DataFrame
    print(df_sample.head())
except FileNotFoundError:
    print("Error: File not found.")
except pd.errors.EmptyDataError:
    print("Error: The file is empty or contains no data.")
except pd.errors.ParserError:
    print("Error: There was an issue parsing the CSV file. Check the format and delimiter.")
except Exception as e:
    print("An unexpected error occurred:", str(e))

  username,anime_id,my_score,user_id,gender,title,type,source,score,scored_by,rank,popularity,genre
0  karthiga,21,9,2255153,Female,One Piece,TV,Mang...                                               
1  karthiga,59,7,2255153,Female,Chobits,TV,Manga,...                                               
2  karthiga,74,7,2255153,Female,Gakuen Alice,TV,M...                                               
3  karthiga,120,7,2255153,Female,Fruits Basket,TV...                                               
4  karthiga,178,7,2255153,Female,Ultra Maniac,TV,...                                               


In [62]:
import pandas as pd
import yaml

# Load the config file
with open('config.yaml', 'r') as config_file:
    config_data = yaml.safe_load(config_file)

try:
    # Read the CSV file based on the config settings
    file_type = config_data['file_type']
    source_file = "./" + config_data['file_name'] + f'.{file_type}'
    df = pd.read_csv(source_file, config_data['inbound_delimiter'])

    # Display the first few rows of the DataFrame
    print(df.head())
except FileNotFoundError:
    print(f"Error: File '{source_file}' not found.")
except pd.errors.EmptyDataError:
    print("Error: The file is empty or contains no data.")
except pd.errors.ParserError:
    print("Error: There was an issue parsing the CSV file. Check the format and delimiter.")
except Exception as e:
    print("An unexpected error occurred:", str(e))



<ipython-input-62-f433b16f7ff3>:12: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df = pd.read_csv(source_file, config_data['inbound_delimiter'])
<ipython-input-62-f433b16f7ff3>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(source_file, config_data['inbound_delimiter'])


  username,anime_id,my_score,user_id,gender,title,type,source,score,scored_by,rank,popularity,genre
0  karthiga,21,9,2255153,Female,One Piece,TV,Mang...                                               
1  karthiga,59,7,2255153,Female,Chobits,TV,Manga,...                                               
2  karthiga,74,7,2255153,Female,Gakuen Alice,TV,M...                                               
3  karthiga,120,7,2255153,Female,Fruits Basket,TV...                                               
4  karthiga,178,7,2255153,Female,Ultra Maniac,TV,...                                               


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 1 columns):
 #   Column                                                                                             Non-Null Count    Dtype 
---  ------                                                                                             --------------    ----- 
 0   username,anime_id,my_score,user_id,gender,title,type,source,score,scored_by,rank,popularity,genre  1048575 non-null  object
dtypes: object(1)
memory usage: 8.0+ MB


In [64]:
import sys
sys.path.append('/path/to/util/directory')


In [65]:
   # Load the config file
with open('config.yaml', 'r') as config_file:
    config_data = yaml.safe_load(config_file)

# Check if 'expected_columns' key exists in config_data dictionary
if 'expected_columns' not in config_data:
    raise ValueError("The 'expected_columns' key is missing in the config file.")

# Continue with the rest of the code for header validation
# ...




In [49]:
import dask.dataframe as dd
from pathlib import Path

In [50]:
%%time
data = dd.read_csv("final_animedataset.csv")

CPU times: user 15.3 ms, sys: 0 ns, total: 15.3 ms
Wall time: 22.3 ms


In [51]:
df

,"username,anime_id,my_score,user_id,gender,title,type,source,score,scored_by,rank,popularity,genre"
0,"karthiga,21,9,2255153,Female,One Piece,TV,Mang..."
1,"karthiga,59,7,2255153,Female,Chobits,TV,Manga,..."
2,"karthiga,74,7,2255153,Female,Gakuen Alice,TV,M..."
3,"karthiga,120,7,2255153,Female,Fruits Basket,TV..."
4,"karthiga,178,7,2255153,Female,Ultra Maniac,TV,..."
...,...
1048570,"Necronomicon,4872,9,118680,Non-Binary,Goku Say..."
1048571,"Necronomicon,4879,8,118680,Non-Binary,Mouryou ..."
1048572,"Necronomicon,4896,6,118680,Non-Binary,Umineko ..."
1048573,"Necronomicon,4898,0,118680,Non-Binary,Kuroshit..."


In [52]:

df.shape

(1048575, 1)

In [53]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 1 columns):
 #   Column                                                                                             Non-Null Count    Dtype 
---  ------                                                                                             --------------    ----- 
 0   username,anime_id,my_score,user_id,gender,title,type,source,score,scored_by,rank,popularity,genre  1048575 non-null  object
dtypes: object(1)
memory usage: 8.0+ MB
